## 인공신경망

### 역전파


#### 다중계층 퍼셉트론 (Multi-Layer Perceptron)
- 단일 퍼셉트론 다중으로 겹쳐 해결가능한 문제 범주를 확대
- 복잡한 입력이 비선형 변경을 통해 분류가 쉬운 은닉 공간으로 매핑

    [참조](https://colah.github.io/posts/2015-09-NN-Types-FP/)

##### MLP 가중치 학습
- 학습으로 최적의 가중치(w) 및 편향(b)를 계산
- 레이어가 복잡할수록 가중치와 bias를 구하기 어려움
- **역전파(Backpropagation) 알고리즘**으로 MLP학습

##### 그래서 역전파란?
- 모델의 잘못된 결과를 예측했다면, 어떤 가중치가 가장 큰 영향을 미쳤는지를 추적한 뒤
- 이 가중치를 조금씩 조정하여 다음에 더 나은 결과를 만들게 하는 것

- 예를 들면, 시험을 보고 틀린 문제의 원인을 분석하는 과정!

##### PyTorch에서는 
- 텐서의 모든 연산에 대한 자동 미분을 제공하며
- 매 순간 적절한 역전파를 제공함

### 모델저장 및 로드

#### DNN(Deep Neural Network)
- 심층신경망 - 여러 비선형 변환의 조합을 통해 높은 수준의 추상화를 시도
    - 은닉계층으로 계속된 비선형 매핑, 목적에 맞는 특징 추출

#### 모델 저장과 로드
- 모델을 이용할 때마다 학습할 필요없음
- 모델 학습시 시간이 많이 소요되므로 매번 학습을 하는 것은 비효율적


#### 함수리스트
- torch.save() - 모델전체(계층 구조, 매개변수 등) 디스크에 저장
- torch.load() - 객체 역직렬화 후 메모리 할당
- state_dict() - 모델의 매개변수(가중치)만을 저장/로드할 때 이용